In [2]:
import sys
import os
# Add the parent directory to the Python path
script_dir = os.getcwd() # Get the current working directory
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'validated_churn_data.parquet')
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
# df[df.select_dtypes(include='object').columns] = df.select_dtypes(include='object').astype('category')

In [4]:
from src.data_split import train_test_split_stratified

X_train, y_train, X_test, y_test = train_test_split_stratified(
    df,
    target_column_name='Exited'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')


X_train.shape=(8000, 10)
y_train.shape=(8000,)
X_test.shape=(2000, 10)
y_test.shape=(2000,)


In [5]:
import lightgbm as lgb
X_train_only_numeric = X_train.select_dtypes(exclude="object")
X_test_only_numeric = X_test.select_dtypes(exclude="object")

In [7]:
model = lgb.LGBMClassifier()
model.fit(X_train_only_numeric, y_train)

[LightGBM] [Info] Number of positive: 1630, number of negative: 6370
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000159 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 850
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203750 -> initscore=-1.363019
[LightGBM] [Info] Start training from score -1.363019


LGBMClassifier()

In [8]:
predictions = model.predict(X_test_only_numeric)
predictions

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [9]:
from sklearn.metrics import accuracy_score,recall_score, f1_score

test_accuracy = accuracy_score(y_test, predictions)
test_recall = recall_score(y_test,predictions )
test_f1 =f1_score(y_test,predictions)
print(f'{test_accuracy=:.4f}')
print(f'{test_recall=:.4f}')
print(f'{test_f1=:.4f}')

test_accuracy=0.8590
test_recall=0.4816
test_f1=0.5816
